### Description
In this notebook, we will see a comprehensive overview of some common tasks that anyone would be implementing in VectorBT Pro. We will be using the [`Double Bollinger Band Strategy`](https://abouttrading.substack.com/p/the-double-bollinger-trading-strategy) as an example of a Multi-Time Frame Strategy to accomplish the following objectives:
* `Resampling` lowertime frame (1m) data to higher time frame data
* Constructing the strategy rules, which uses two different timeframes for the entry and exit conditions
* Visualize the indicators and entry/exit signals on VBT Plots
* Run the backtesting simulation using `from_signals`
* Visualize and discuss the simulation results

In [ ]:
import pandas as pd
import numpy as np
import vectorbtpro as vbt

In [ ]:
## Load m1 data
m1_data = vbt.HDFData.fetch('../../data/GU_OHLCV_3Y.h5')
m1_data.wrapper.index #pandas doaesn't recognise the frequency because of missing timestamps

Downsampling `1 minute` data to Higher Time Frames

In [ ]:
m5_data = m1_data.resample('5T')   # Convert 1 minute to 5 mins
m15_data = m1_data.resample('15T') # Convert 1 minute to 15 mins
m30_data = m1_data.resample('30T') # Convert 1 minute to 30 mins
h1_data = m1_data.resample("1h")   # Convert 1 minute to 1 hour
h4_data = m1_data.resample("4h")   # Convert 1 minute to 4 hour
d1_data = m1_data.resample("1d")   # Convert 1 minute to daily

Check missing frequency on the resampled higher timeframe data.
- The missing frequency has been checked for ` freq = '4H'` `  

In [ ]:
h4_data.wrapper.index

In [ ]:
# Obtain all the closing  prices using the .get() method
m5_close = m5_data.get()['Close']
m30_close = m30_data.get()['Close']

## h1 data
h1_open =  h1_data.get()['Open']
h1_close = h1_data.get()['Close']
h1_high =  h1_data.get()['High']
h1_low =  h1_data.get()['Low']

## h4 data
h4_open  = h4_data.get()['Open']
h4_close  = h4_data.get()['Close']
h4_high  = h4_data.get()['High']
h4_low  = h4_data.get()['Low']

In [ ]:
## Check for a sample of missing time periods during weelemds
## Looks like a typical closure during weekends, on Friday evening (30.08.2019) and reopening at Tokyo market open
# m30_close[(m30_close.index > "2019-08-29") & (m30_close.index < "2019-09-02")]
h4_close[(h4_close.index > "2019-08-29") & (h4_close.index < "2019-09-02")]#.dropna()

In [ ]:
h4_close.info()

### Create Indicators for multiple timeframes

This _Double Bollinger Band_ strategy assumes a double confirmation factor from two types of Bollinger bands. Basically, to create the strategy, you need to do the following:
* Calculate a 20-period RSI on the market price.
* Calculate a 20-period Bollinger bands (with 2x standard deviation) on the market price.
* Calculate a 20-period Bollinger bands (with 2x standard deviation) on the RSI.

**TODO** :
* Find out what the `.real` does?

In [ ]:
rsi_period = 21

m5_rsi = vbt.talib("RSI", timeperiod = rsi_period).run(m5_data.get("Close"), skipna=True).real
m5_bbands = vbt.talib("BBANDS").run(m5_data.get("Close"), skipna=True)
m5_bbands_rsi = vbt.talib("BBANDS").run(m5_rsi, skipna=True)

m30_rsi = vbt.talib("RSI", timeperiod = rsi_period).run(m30_data.get("Close"), skipna=True).real
m30_bbands = vbt.talib("BBANDS").run(m30_data.get("Close"), skipna=True)
m30_bbands_rsi = vbt.talib("BBANDS").run(m30_rsi, skipna=True)

h1_rsi = vbt.talib("RSI", timeperiod = rsi_period).run(h1_data.get("Close"), skipna=True).real
h1_bbands = vbt.talib("BBANDS").run(h1_data.get("Close"), skipna=True)
h1_bbands_rsi = vbt.talib("BBANDS").run(h1_rsi, skipna=True)

h4_rsi = vbt.talib("RSI", timeperiod = rsi_period).run(h4_data.get("Close"), skipna=True).real
h4_bbands = vbt.talib("BBANDS").run(h4_data.get("Close"), skipna=True)
h4_bbands_rsi = vbt.talib("BBANDS").run(h4_rsi, skipna=True)

In [ ]:
## Alternate fancier method 2 of constructing BBAnd data for MTF data
rsi = vbt.talib("RSI", timeperiod = rsi_period).run(m5_close, skipna=True,
                                                   timeframe=["5T","30T","1h","4h"],
                                                   broadcast_kwargs=dict(wrapper_kwargs=dict(freq="5T"))).real
                                                   
bbands_price = vbt.talib("BBANDS").run(m5_close, skipna=True, 
                                timeframe=["5T","30T","1h","4h"], 
                                broadcast_kwargs=dict(wrapper_kwargs=dict(freq="5T")))


bbands_rsi = vbt.talib("BBANDS").run(m5_rsi, skipna=True, 
                                timeframe=["5T","30T","1h","4h"], 
                                broadcast_kwargs=dict(wrapper_kwargs=dict(freq="5T")))

In [ ]:
m5_bbands.lowerband.info()

In [ ]:
bbands_price['5T'].lowerband.info()

In [ ]:
## Extracting the bbands_data
pd.DataFrame(data = {"5m_bb_price_lower" : bbands_price["5T"].lowerband,
                     "5m_bb_price_middle" : bbands_price["5T"].middleband,
                     "5m_bb_price_higher": bbands_price["5T"].upperband})


In [ ]:
pd.DataFrame(data = {"h4_bb_price_lower" : bbands_price["4h"].lowerband, 
                    "h4_bb_price_middle" : bbands_price["4h"].middleband, 
                    "h4_bb_price_higher": bbands_price["4h"].upperband}
                    )


Upsampling higher timeframe (`low frequency`) data to lower timeframe (`high frequency`) data

In [ ]:
def create_resamplers(result_dict_keys_list : list, source_indices : list,  source_frequencies :list, target_index : pd.Series, target_freq : str):
    """
    Creates a dictionary of vbtpro resampler objects.

    Parameters
    ==========
    result_dict_keys_list : list, list of strings, which are keys of the output dictionary
    source_indices        : list, list of pd.time series objects of the higher timeframes
    source_frequencies    : list, list of strings, which are short form representation of the time series order. Eg:["1D", "4h"]
    target_index          : pd.Series, target time series for the resampler objects
    target_freq           : str, target time frequency for the resampler objects, i.e.
                                 the 1st frequency at the start of `source_frequencies` list

    Returns
    ===========
    
    resamplers_dict       : dict, vbt pro resampler objects
    
    """
    
    
    resamplers = []
    for si, sf in zip(source_indices, source_frequencies):
        resamplers.append(vbt.Resampler(source_index = si,  target_index = target_index,  source_freq=sf, target_freq=target_freq))
    return dict(zip(result_dict_keys_list, resamplers))

In [ ]:
## Create Resampler Objects for upsampling
src_indices = [m30_close.index, h1_close.index, h4_close.index]
src_frequencies = ["30T", "1h", "4h"]
resampler_dict_keys = ["m30_m5", "h1_m5", "h4_m5"]

list_resamplers = create_resamplers(resampler_dict_keys, src_indices, src_frequencies, m5_close.index, "5T")
list_resamplers

In [ ]:
# series_to_resample = [
#     [m30_close,rsi['30T'], bbands_price['30T'].upperband, bbands_price['30T'].middleband, bbands_price['30T'].lowerband, bbands_rsi['30T'].upperband, bbands_rsi['30T'].middleband, bbands_rsi['30T'].lowerband],
#     [h1_close, rsi['1h'], bbands_price['1h'].upperband, bbands_price['1h'].middleband, bbands_price['1h'].lowerband, bbands_rsi['1h'].upperband, bbands_rsi['1h'].middleband, bbands_rsi['1h'].lowerband],
#     [h4_close, rsi['4h'], bbands_price['4h'].upperband, bbands_price['4h'].middleband, bbands_price['4h'].lowerband, bbands_rsi['4h'].upperband, bbands_rsi['4h'].middleband, bbands_rsi['4h'].lowerband]
#     ]

series_to_resample = [
    [m30_close, m30_rsi, m30_bbands.upperband, m30_bbands.middleband, m30_bbands.lowerband, m30_bbands_rsi.upperband, m30_bbands_rsi.middleband, m30_bbands_rsi.lowerband],
    [h1_close, h1_rsi, h1_bbands.upperband, h1_bbands.middleband, h1_bbands.lowerband, h1_bbands_rsi.upperband, h1_bbands_rsi.middleband, h1_bbands_rsi.lowerband],
    [h4_close, h4_rsi, h4_bbands.upperband, h4_bbands.middleband, h4_bbands.lowerband, h4_bbands_rsi.upperband, h4_bbands_rsi.middleband, h4_bbands_rsi.lowerband]
    ]

data_keys = [
        ["m30_close", "m30_rsi", "m30_bband_price_upper",  "m30_bband_price_middle", "m30_bband_price_lower",  "m30_bband_rsi_upper",  "m30_bband_rsi_middle", "m30_bband_rsi_lower"],
        ["h1_close", "h1_rsi", "h1_bband_price_upper",  "h1_bband_price_middle",  "h1_bband_price_lower",  "h1_bband_rsi_upper",  "h1_bband_rsi_middle", "h1_bband_rsi_lower" ],
        ["h4_close", "h4_rsi", "h4_bband_price_upper",  "h4_bband_price_middle",  "h4_bband_price_lower",  "h4_bband_rsi_upper",  "h4_bband_rsi_middle", "h4_bband_rsi_lower" ]
         ]

### Resample and create multi-timeframe data frame
* Our baseline time-frame (frequency) is 5min data

In [ ]:
data = {"m5_close" : m5_close.ffill(), "m5_rsi" : m5_rsi.ffill(), 
        "m5_bband_price_upper" : m5_bbands.upperband.ffill(),  "m5_bband_price_middle" : m5_bbands.middleband.ffill(),  "m5_bband_price_lower" : m5_bbands.lowerband.ffill(),
        "m5_bband_rsi_upper" : m5_bbands_rsi.upperband.ffill(),  "m5_bband_rsi_middle" : m5_bbands_rsi.middleband.ffill(), "m5_bband_rsi_lower" : m5_bbands_rsi.lowerband.ffill() 
        }

In [ ]:
for lst_series, lst_keys in zip(series_to_resample, data_keys):
    for key, time_series in zip(lst_keys, lst_series):
        data[key] = time_series#.ffill()

In [ ]:
%%time

# for lst_series, lst_keys, resampler in zip(series_to_resample, data_keys, resampler_dict_keys):
#     for key, time_series in zip(lst_keys, lst_series):
#         resampled_time_series = time_series.vbt.resample_closing(list_resamplers[resampler])
#         data[key] = resampled_time_series.ffill()

In [ ]:
## Add h1 data
data["h1_open"] = h1_open.vbt.resample_closing(list_resamplers['h1_m5']).ffill()
data["h1_low"]  = h1_low.vbt.resample_closing(list_resamplers['h1_m5']).ffill()
data["h1_high"] = h1_high.vbt.resample_closing(list_resamplers['h1_m5']).ffill()

## Add h4 data
data["h4_open"] = h4_open.vbt.resample_closing(list_resamplers['h4_m5']).ffill()
data["h4_low"]  = h4_low.vbt.resample_closing(list_resamplers['h4_m5']).ffill()
data["h4_high"] = h4_high.vbt.resample_closing(list_resamplers['h4_m5']).ffill()
# data

In [ ]:
print(data["h4_rsi"].equals(other = rsi["4h"]) )
print(data["m5_rsi"].equals(other = rsi["5T"]) )
print(data["m5_bband_price_lower"].equals(other = bbands_price["5T"].lowerband) )

In [ ]:
# print(h4_rsi.ffill().equals(other = rsi["4h"]) )
# print(m5_bbands.lowerband.equals(other = bbands_price["5T"].lowerband) )
# print(m5_bbands.lowerband.ffill().equals(other = bbands_price["5T"].lowerband) )

### Construct dataframe of multi-time frame data

In [ ]:
## construct a multi-timeframe dataframe
mtf_df = pd.DataFrame(data)
print("Length of mtf_df:",len(mtf_df))

In [ ]:
display(mtf_df)

In [ ]:
mtf_df.info()

Double Bollinger Band - **Strategy Conditions**

The trading conditions (rules) of the strategy are as follows:
1. A long (buy) signal is generated whenever the market price surpasses its, lower Bollinger band after having been below it while simultaneously, the RSI surpasses its lower Bollinger band after having been below it.
2. A short (sell) signal is generated whenever the market price breaks its upper Bollinger band after having been above it while simultaneously, the RSI breaks its upper Bollinger band after having been above it.

In [ ]:
required_cols = ['m5_close', "m5_rsi" , "m5_bband_rsi_lower", "h1_low",'h4_low', "h1_rsi" ,"h4_rsi", "h1_bband_price_lower" ,"h4_bband_price_lower"]

In [ ]:
mtf_df[required_cols][(mtf_df['h4_low'] < mtf_df['h4_bband_price_lower'])]

**Checking for entries on 5m chart**

In [ ]:
## Higher values greater than 1.0 are like moving up the lower RSI b-band, signifying if rsi is anywhere around 1% of the lower b-band validate that case as True
bb_upper_fract = 0.99
bb_lower_fract = 1.01

In [ ]:
## Yields very few results
mtf_df[required_cols][(mtf_df['m5_rsi'] < (bb_lower_fract * mtf_df['m5_bband_rsi_lower']) ) & (mtf_df['m5_rsi'] <= 30)]

In [ ]:
mtf_df[required_cols][(mtf_df['m5_rsi'] < (bb_lower_fract * mtf_df['m5_bband_rsi_lower']) )]

In [ ]:
df_tmp = mtf_df[required_cols][(mtf_df['h4_low'] <= mtf_df['h4_bband_price_lower']) & (mtf_df['m5_rsi'] <= (bb_lower_fract * mtf_df['m5_bband_rsi_lower'])) ]
print("Nr. of rows satisfying LONG condition(s):",len(df_tmp))
df_tmp

In [ ]:
mtf_df.columns

In [ ]:
## Using m5 and h4 data 
mtf_df['long_entry']=(mtf_df['h4_low'] <= mtf_df['h4_bband_price_lower']) & (mtf_df['m5_rsi'] <= (bb_lower_fract * mtf_df['m5_bband_rsi_lower']) )
mtf_df['long_exit']=(mtf_df['h4_high'] >= mtf_df['h4_bband_price_upper']) & (mtf_df['m5_rsi'] >= (bb_upper_fract * mtf_df['m5_bband_rsi_upper']) )

mtf_df['signal'] = 0   
mtf_df['signal'] = np.where( mtf_df['long_entry'] ,1, 0)
mtf_df['signal'] = np.where( mtf_df['long_exit'] ,-1, mtf_df['signal'])
mtf_df.head()

In [ ]:
long_entries = mtf_df.signal == 1.0
long_exits = mtf_df.signal == -1.0

mtf_df['short_entries'] = short_entries = long_exits
mtf_df['short_exits'] = short_exits = long_entries

**Resample `entries` and `exits` from `5m` to `H4`**

In [ ]:
print(long_entries.vbt.signals.total(),long_exits.vbt.signals.total(),short_entries.vbt.signals.total(), short_exits.vbt.signals.total())
print(len(long_entries), len(long_exits), len(short_entries), len(short_exits))

In [ ]:
## Clean redundant and duplicate signals
clean_long_entries, clean_long_exits = long_entries.vbt.signals.clean(long_exits)
clean_short_entries, clean_short_exits = short_entries.vbt.signals.clean(short_exits)

In [ ]:
print(clean_long_entries.vbt.signals.total(),clean_long_exits.vbt.signals.total(), 
      clean_short_entries.vbt.signals.total(),clean_short_exits.vbt.signals.total())
print(len(clean_long_entries), len(clean_long_exits), len(clean_short_entries), len(clean_short_exits))

In [ ]:
%%time
## Slower Method
# h4_long_entries = long_entries.resample("4h").apply(lambda array: np.any(array))
# h4_long_exits = long_exits.resample("4h").apply(lambda array: np.any(array))
# h4_short_entries = short_entries.resample("4h").apply(lambda array: np.any(array))
# h4_short_exits = short_exits.resample("4h").apply(lambda array: np.any(array))

In [ ]:
%%time
## Faster method
h4_long_entries = long_entries.vbt.resample_apply("4h", "any", wrap_kwargs=dict(dtype=bool))
h4_long_exits = long_exits.vbt.resample_apply("4h", "any", wrap_kwargs=dict(dtype=bool))
h4_short_entries = short_entries.vbt.resample_apply("4h", "any", wrap_kwargs=dict(dtype=bool))
h4_short_exits = short_exits.vbt.resample_apply("4h", "any", wrap_kwargs=dict(dtype=bool))

In [ ]:
print(len(h4_long_entries),len(h4_long_exits),len(h4_short_entries),len(h4_short_exits))
print(h4_long_entries.vbt.signals.total(),h4_long_exits.vbt.signals.total(),
      h4_short_entries.vbt.signals.total(),h4_short_exits.vbt.signals.total())

In [ ]:
clean_h4_long_entries, clean_h4_long_exits = h4_long_entries.vbt.signals.clean(h4_long_exits)
clean_h4_short_entries, clean_h4_short_exits = h4_short_entries.vbt.signals.clean(h4_short_exits)

In [ ]:
print(len(clean_h4_long_entries),len(clean_h4_long_exits),len(clean_h4_short_entries),len(clean_h4_short_exits))
print(clean_h4_long_entries.vbt.signals.total(),clean_h4_long_exits.vbt.signals.total(), \
      clean_h4_short_entries.vbt.signals.total(),clean_h4_short_exits.vbt.signals.total())

In [ ]:
## Combine long and short entries/exits into a single series
entries = pd.Series(h4_long_entries.values | h4_short_entries.values, index=h4_long_entries.index) #.sort_index()#.duplicated(keep='last')
exits = pd.Series(h4_long_exits.values | h4_short_exits.values, index = h4_long_exits.index) #.sort_index()#.duplicated(keep='last')
print(len(entries), len(exits))
print(entries.vbt.signals.total(),exits.vbt.signals.total())

#### Plot Indicators
Stage-wise plotting

In [ ]:
## Global Plot Settings
vbt.settings.set_theme("dark")
vbt.settings['plotting']['layout']['width'] = 1280

In [ ]:
h4_df = h4_data.get()
h4_df

In [ ]:
## Plot OHLCV data first
kwargs1 = {"title_text" : "OHLCV Plot", "title_font_size" : 18}
h4_ohlc_sample = h4_df[["Open", "High", "Low", "Close"]].iloc[100:200]#.dropna()
f = h4_ohlc_sample.vbt.ohlcv.plot(**kwargs1)
f.show()

In [ ]:
kwargs1 = {"title_text" : "OHLCV Plot with BBands", "title_font_size" : 18}
## child figure inherits the parent figure's style and settings from the **kwargs
h4_bbands.iloc[100:200].plot(fig = f,
                            lowerband_trace_kwargs=dict(fill=None, name = 'BB_Price_Lower'), 
                            upperband_trace_kwargs=dict(fill=None, name = 'BB_Price_Upper'),
                            middleband_trace_kwargs=dict(fill=None, name = 'BB_Price_Middle')).show()


In [ ]:
kwargs2 = {"title_text" : "H4 BB_RSI", "title_font_size" : 18 }
h4_bbands_rsi.iloc[100:200].plot(xaxis=dict(rangeslider_visible=True),limits=(25, 75),
                                lowerband_trace_kwargs=dict(fill=None, name = 'BB_RSI_Lower'),
                                upperband_trace_kwargs=dict(fill=None, name = 'BB_RSI_Upper'),
                                middleband_trace_kwargs=dict(fill=None, name = 'BB_RSI_Middle'),
                                **kwargs2).show()

#### Stacked SubPlots on sliced data

In [ ]:
# retrieve the dates that are in the original datset
dt_obs = h4_df.index.to_list()
# Drop rows with missing values
dt_obs_dropped = h4_df['Close'].dropna().index.to_list()
# store  dates with missing values
dt_breaks = [d for d in dt_obs if d not in dt_obs_dropped]
len(dt_obs), len(dt_obs_dropped), len(dt_breaks)

In [ ]:
kwargs1 = {"title_text" : "H4 OHLCV with BBands on Price and RSI", "title_font_size" : 18, 
           "legend" : dict(yanchor="top",y=0.99, xanchor="right",x= 0.25)}

fig = vbt.make_subplots(rows=2,cols=1, shared_xaxes=True, vertical_spacing=0.1)

## Sliced Data
h4_price = h4_df[["Open", "High", "Low", "Close"]]
indices = slice(100,200)
h4_price.iloc[indices].vbt.ohlcv.plot(add_trace_kwargs=dict(row=1, col=1),  fig=fig, **kwargs1) 
h4_bbands.iloc[indices].plot(add_trace_kwargs=dict(row=1, col=1),fig=fig,
                            lowerband_trace_kwargs=dict(fill=None, name = 'BB_Price_Lower'), 
                            upperband_trace_kwargs=dict(fill=None, name = 'BB_Price_Upper'),
                            middleband_trace_kwargs=dict(fill=None, name = 'BB_Price_Middle'))

h4_rsi.iloc[indices].rename("RSI").vbt.plot(add_trace_kwargs=dict(row=2, col=1),fig=fig, **kwargs1 )

h4_bbands_rsi.iloc[indices].plot(add_trace_kwargs=dict(row=2, col=1),limits=(25, 75),fig=fig,
                            lowerband_trace_kwargs=dict(fill=None, name = 'BB_RSI_Lower'), 
                            upperband_trace_kwargs=dict(fill=None, name = 'BB_RSI_Upper'),
                            middleband_trace_kwargs=dict(fill=None, name = 'BB_RSI_Middle'),
                            # xaxis=dict(rangeslider_visible=True) ## Without Range Slider
                            )

fig.update_xaxes(rangebreaks=[dict(values=dt_breaks)])
fig.layout.showlegend = False
fig.show_svg()

**Functionize the complete plot for 2BB Strategy**

In [ ]:
def stacked_2bb_rsi_plot(slice_lower : str, slice_upper: str, df : pd.DataFrame , rsi : pd.Series,
                         bb_price : vbt.indicators.factory, bb_rsi : vbt.indicators.factory, 
                         long_entries: pd.Series, long_exits: pd.Series, 
                         short_entries: pd.Series, short_exits: pd.Series,
                         show_legend : bool = True):
    """Creates a stacked indicator plot for the 2BB strategy.
    Parameters
    ===========
    slice_lower : str, start date of dataframe slice in yyyy.mm.dd format
    slice_upper : str, start date of dataframe slice in yyyy.mm.dd format
    df          : pd.DataFrame, containing the OHLCV data
    rsi         : pd.Series, rsi indicator time series in same freq as df
    bb_price    : vbt.indicators.factory.talib('BBANDS'), computed on df['close'] price
    bb_rsi      : vbt.indicators.factory.talib('BBANDS') computer on RSI
    long_entries: pd.Series, time series data of long entries
    long_exits  : pd.Series, time series data of long exits
    short_entries: pd.Series, time series data of short entries
    short_exits : pd.Series, time series data of short exits
    show_legend : bool, switch to show or completely hide the legend box on the plot
    
    Returns
    =======
    fig         : plotly figure object
    """
    kwargs1 = {"title_text" : "H4 OHLCV with BBands on Price and RSI", 
               "title_font_size" : 18,
               "height" : 960,
               "legend" : dict(yanchor="top",y=0.99, xanchor="right",x= 0.1)}
    fig = vbt.make_subplots(rows=2,cols=1, shared_xaxes=True, vertical_spacing=0.1)
    ## Filter Data according to date slice
    df_slice = df[["Open", "High", "Low", "Close"]][slice_lower : slice_upper]
    bb_price = bb_price[slice_lower : slice_upper]
    rsi = rsi[slice_lower : slice_upper]
    bb_rsi = bb_rsi[slice_lower : slice_upper]
    long_entries = long_entries[slice_lower : slice_upper]
    long_exits = long_exits[slice_lower : slice_upper]
    short_entries = short_entries[slice_lower : slice_upper]
    short_exits = short_exits[slice_lower : slice_upper]    

    ## Retrieve datetime index of rows where price data is NULL
    # retrieve the dates that are in the original datset
    dt_obs = df_slice.index.to_list()
    # Drop rows with missing values
    dt_obs_dropped = df_slice['Close'].dropna().index.to_list()
    # store  dates with missing values
    dt_breaks = [d for d in dt_obs if d not in dt_obs_dropped]

    ## Plot Figures
    df_slice.vbt.ohlcv.plot(add_trace_kwargs=dict(row=1, col=1),  fig=fig, **kwargs1) ## Without Range Slider
    rsi.rename("RSI").vbt.plot(add_trace_kwargs=dict(row=2, col=1), trace_kwargs = dict(connectgaps=True), fig=fig, **kwargs1 ) 

    bb_price.plot(add_trace_kwargs=dict(row=1, col=1),fig=fig, **kwargs1,
                    lowerband_trace_kwargs=dict(fill=None, name = 'BB_Price_Lower', connectgaps=True), 
                    upperband_trace_kwargs=dict(fill=None, name = 'BB_Price_Upper', connectgaps=True),
                    middleband_trace_kwargs=dict(fill=None, name = 'BB_Price_Middle', connectgaps=True) )

    bb_rsi.plot(add_trace_kwargs=dict(row=2, col=1),limits=(25, 75),fig=fig,
                lowerband_trace_kwargs=dict(fill=None, name = 'BB_RSI_Lower', connectgaps=True), 
                upperband_trace_kwargs=dict(fill=None, name = 'BB_RSI_Upper', connectgaps=True),
                middleband_trace_kwargs=dict(fill=None, name = 'BB_RSI_Middle', connectgaps=True))

    ## Add Long Entries and Long Exits
    long_entries.vbt.signals.plot_as_entries(bb_price.lowerband,fig=fig,
                                            trace_kwargs=dict(marker=dict(color="limegreen"), 
                                                              name="Long entries")
                                            )
    long_exits.vbt.signals.plot_as_exits(bb_price.upperband, fig = fig,
                                        trace_kwargs=dict(marker=dict(color="red"), 
                                                          name="Long exits"), 
                                        )
    long_entries.vbt.signals.plot_as_entries(bb_rsi.lowerband, fig = fig,
                                             add_trace_kwargs=dict(row=2, col=1),
                                             trace_kwargs=dict(marker=dict(color="limegreen"),
                                                               showlegend = False))  

    long_exits.vbt.signals.plot_as_exits(bb_rsi.upperband, fig = fig,showlegend = False,
                                         add_trace_kwargs=dict(row=2, col=1),
                                         trace_kwargs=dict(marker=dict(color="red"),
                                                           showlegend = False))

    ## Add Short Entries and Short Exits
    shot_entry_marker_style = dict(color="lightgreen",symbol="circle-open",size = 15)
    short_exit_marker_style = dict(color="lightpink", symbol="circle-open", size = 15)
    
    short_entries.vbt.signals.plot_as_entries(bb_price.upperband, fig=fig,
                                              trace_kwargs=dict(name="Short entries",
                                                                marker=shot_entry_marker_style,
                                              ))
    short_exits.vbt.signals.plot_as_exits(bb_price.lowerband, fig = fig,
                                          trace_kwargs=dict(name="Short exits",
                                                            marker=short_exit_marker_style,
                                          ))
    short_entries.vbt.signals.plot_as_entries(bb_rsi.upperband, fig = fig,
                                              add_trace_kwargs=dict(row=2, col=1),
                                              trace_kwargs=dict(marker = shot_entry_marker_style,
                                                                showlegend = False)
                                              )  
                                                
    short_exits.vbt.signals.plot_as_exits(bb_rsi.lowerband,fig = fig,
                                          add_trace_kwargs=dict(row=2, col=1),
                                          trace_kwargs=dict(marker = short_exit_marker_style,
                                                            showlegend = False)
                                          )      

    fig.update_xaxes(rangebreaks=[dict(values=dt_breaks)])
    fig.layout.showlegend = show_legend  
    fig.write_html(f"2BB_Strategy_{slice_lower}_to_{slice_upper}.html")
    
    return fig

In [ ]:
slice_lower = '2019.10.01'
slice_higher = '2019.10.30'
fig = stacked_2bb_rsi_plot(slice_lower, slice_higher, h4_df, h4_rsi, 
                           h4_bbands, h4_bbands_rsi, 
                           clean_h4_long_entries, clean_h4_long_exits, 
                           clean_h4_short_entries, clean_h4_short_exits,
                           show_legend = True)
fig.show()

### Create Backtest Simulation for 2BB Strategy using `vbt.Portfolio.from_signals()`

In [ ]:
pf_2_bb_custom = vbt.Portfolio.from_signals(
    mtf_df['m5_close'], 
    entries=long_entries, 
    exits=long_exits, 
    short_entries=short_entries, 
    short_exits=short_exits, 
    freq=pd.Timedelta(minutes=5), 
    init_cash=100000
)

In [ ]:
pf_2_bb_custom.stats()

In [ ]:
# pf_2_bb_custom.plot().show() ## This takes too long (30 secs) as it uses 5m timeframe
pf_2_bb_custom.resample("1d").plot().show()

In [ ]:
# pf_2_bb_custom.orders.plot(xaxis=dict(rangeslider_visible=True),**kwargs1).show()
pf_2_bb_custom.orders.resample("1d").plot(xaxis=dict(rangeslider_visible=True),**kwargs1).show()

In [ ]:
## Simple line plots seem to be using matplotlib
pf_2_bb_custom.trades.records_readable.PnL.cumsum().plot()

In [ ]:
pf_2_bb_custom.positions.records_readable

In [ ]:
pf_2_bb_custom.trades.records_readable

In [ ]:
pnl_series = pf_2_bb_custom.trades.pnl.to_pd()
pnl_series[pnl_series.notna()]

In [ ]:
signal = mtf_df['signal'].copy()
signal.loc[clean_long_entries] = 2
signal.loc[clean_long_exits] = 1
signal.loc[clean_short_entries] = -2
signal.loc[clean_short_exits] = -1

In [ ]:
mtf_df.signal[mtf_df['signal'] != 0]

In [ ]:
signal[signal != 0]

In [ ]:
mtf_df['cleaned_signal'] = signal

In [ ]:
mtf_df["signal"].equals(mtf_df['cleaned_signal'])

In [ ]:
mtf_df["pnl"] = pnl_series
mtf_df

In [ ]:
mtf_df.to_csv("2BB_Final_Simulation.csv", index = False)

In [ ]:
mtf_df.columns

**TODO**
* Can we add dotted lines in the plot to connect an entry and exit of a trade order?
* How to access floating equity and balance after the simulation or it the same as `PnL`?
* What other type of order analytics can we do to make this notebook more complete / informative?